# XGBoost Demo - GPUs

# Setup the data
We generate some mock data and create training and validation sets

In [8]:
# The version of xgboost on this VM has been
# compiled to take advantage of GPUs
library('xgboost')

# Simulate N x p random matrix with some 
# binomial response dependent on pp columns

# 2 Million x 50 matrix
N <- 2000000
p <- 50
pp <- 25

pt <- proc.time()

X <- matrix(runif(N * p), ncol = p)
betas <- 2 * runif(pp) - 1
sel <- sort(sample(p, pp))
m <- X[, sel] %*% betas - 1 + rnorm(N)
y <- rbinom(N, 1, plogis(m))

# Create training & validation sets
tr <- sample.int(N, N * 0.75)
dtrain <- xgb.DMatrix(X[tr,], label = y[tr])
dtest <- xgb.DMatrix(X[-tr,], label = y[-tr])
wl <- list(train = dtrain, test = dtest)

elapsed_time <- (proc.time() - pt)[["elapsed"]]
cat("\nElapsed time = ", elapsed_time, " seconds")


Elapsed time =  8.368  seconds

# GPUs - Start training!

The only difference between the CPU version of this code and the GPU version is that the tree_method changes from 'hist' to 'gpu_hist'.  Everything else is identical

In [7]:
# An example of running 'gpu_hist' algorithm
# which is
# - similar to the 'hist'
# - the fastest option for moderately large datasets
# - current limitations: max_depth < 16, does not 
#   implement guided loss

pt <- proc.time()

param <- list( 
              objective = 'binary:logistic', 
              eval_metric = 'auc', 
              tree_method = 'gpu_hist' # use GPU optimized histogram method
             )

bst <- xgb.train( 
                 param, 
                 dtrain, 
                 nrounds = 50,
                )

print("Training complete")

elapsed_time <- (proc.time() - pt)[["elapsed"]]
cat("\nElapsed time = ", elapsed_time, " seconds")


[1] "Training complete"

Elapsed time =  1.308  seconds